In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# --- 0. 定義檔案路徑 ---
# 在 Colab 中，檔案會存在當前目錄
INPUT_CSV = 'multidimensional_analysis_results_SLM+人工_不含理由_刪除無效_刪除評論_標準名稱_20251112.csv'
OUTPUT_EXCEL = 'model_consistency_analysis_results.xlsx'

print(f"分析開始...")
print(f"讀取資料: {INPUT_CSV}")

# --- 1. 定義評級映射 (Rating Maps) ---
sentiment_map = {
    '非常正面': 5, '正面': 4, '中立': 3, '負面': 2, '非常負面': 1
}
sarcasm_map = {
    '有反諷': 1, '未提及': 0
}
six_level_map = {
    '非常好': 6, '好': 5, '一般': 4, '差': 3, '非常差': 2, '未提及': 1
}

DIMENSION_MAPS = {
    'Column1.analysis.sentiment.level': sentiment_map,
    'Column1.analysis.sarcasm.level': sarcasm_map,
    'Column1.analysis.comfort.level': six_level_map,
    'Column1.analysis.value.level': six_level_map,
    'Column1.analysis.facilities.level': six_level_map,
    'Column1.analysis.location.level': six_level_map,
    'Column1.analysis.staff.level': six_level_map,
    'Column1.analysis.cleanliness.level': six_level_map,
    'Column1.analysis.food.level': six_level_map
}
DIMENSION_LABELS = {dim: list(map.keys()) for dim, map in DIMENSION_MAPS.items()}
dimensions_to_analyze = list(DIMENSION_MAPS.keys())

# --- 2. 定義 Off-by-1 計算函式 ---
def calculate_off_by_1(y_true, y_pred, dimension_name):
    if dimension_name == 'Column1.analysis.sarcasm.level':
        return np.nan

    current_map = DIMENSION_MAPS[dimension_name]
    y_true_num = y_true.map(current_map)
    y_pred_num = y_pred.map(current_map)

    valid_indices = y_true_num.notna() & y_pred_num.notna()
    if valid_indices.sum() == 0:
        return np.nan

    diff = (y_true_num[valid_indices] - y_pred_num[valid_indices]).abs()
    correct = (diff <= 1).sum()
    total = valid_indices.sum()
    return correct / total if total > 0 else np.nan

# --- 3. 讀取並準備資料 ---
try:
    df = pd.read_csv(INPUT_CSV)

    human_df = df[df['Column1.model'] == '人工標註'].set_index('Column1.key')
    models_df = df[df['Column1.model'] != '人工標註']
    model_names = models_df['Column1.model'].unique()
    total_human_reviews = human_df.index.nunique()

    print(f"基準 (人工標註) 評論總數: {total_human_reviews}")

    # --- 4. 計算完成率 (Completion Rate) ---
    print("正在計算完成率...")
    completion_counts = models_df.groupby('Column1.model')['Column1.key'].nunique()
    completion_df = pd.DataFrame(completion_counts).reset_index()
    completion_df.columns = ['model', 'Completed_Reviews']
    completion_df['Total_Human_Reviews'] = total_human_reviews
    completion_df['Completion_Rate_Percent'] = (completion_df['Completed_Reviews'] / total_human_reviews) * 100
    completion_df = completion_df.sort_values(by='Completion_Rate_Percent', ascending=False).set_index('model')

    # --- 5. 執行一致性分析 (Accuracy, Macro-F1, Off-by-1) ---
    print("正在計算一致性指標 (Accuracy, F1, Off-by-1)...")
    all_results = []
    for model_name in model_names:
        current_model_df = models_df[models_df['Column1.model'] == model_name].set_index('Column1.key')
        merged_df = human_df[dimensions_to_analyze].join(current_model_df[dimensions_to_analyze], lsuffix='_human', rsuffix='_model')
        merged_df.dropna(inplace=True) # 只比較雙方都有的

        for dim in dimensions_to_analyze:
            y_true = merged_df[f'{dim}_human']
            y_pred = merged_df[f'{dim}_model']

            if y_true.empty:
                acc, macro_f1, off_by_1 = np.nan, np.nan, np.nan
            else:
                current_labels = DIMENSION_LABELS[dim]
                acc = accuracy_score(y_true, y_pred)
                macro_f1 = f1_score(y_true, y_pred, labels=current_labels, average='macro', zero_division=0)
                off_by_1 = calculate_off_by_1(y_true, y_pred, dim)

            all_results.append({'model': model_name, 'dimension': dim, 'metric': 'Accuracy', 'score': acc})
            all_results.append({'model': model_name, 'dimension': dim, 'metric': 'Macro-F1', 'score': macro_f1})
            all_results.append({'model': model_name, 'dimension': dim, 'metric': 'Off-by-1', 'score': off_by_1})

    results_df = pd.DataFrame(all_results)
    results_df['dimension'] = results_df['dimension'].str.replace('Column1.analysis.', '', regex=False).str.replace('.level', '', regex=False)

    pivot_accuracy = results_df[results_df['metric'] == 'Accuracy'].pivot(index='model', columns='dimension', values='score')
    pivot_macro_f1 = results_df[results_df['metric'] == 'Macro-F1'].pivot(index='model', columns='dimension', values='score')
    pivot_off_by_1 = results_df[results_df['metric'] == 'Off-by-1'].pivot(index='model', columns='dimension', values='score')

    # --- 6. 計算總體 (懲罰性) 指標 ---
    print("正在計算總體 (懲罰性) 指標...")
    completion_ratio_series = completion_df['Completion_Rate_Percent'] / 100
    overall_accuracy_df = pivot_accuracy.multiply(completion_ratio_series, axis=0)
    overall_off_by_1_df = pivot_off_by_1.multiply(completion_ratio_series, axis=0)

    # --- 7. (*** 新增 ***) 定義您要的欄位順序 ---
    final_column_order = [
        'sentiment',
        'sarcasm',
        'comfort',
        'value',
        'facilities',
        'location',
        'staff',
        'cleanliness',
        'food'
    ]

    # 檢查以防萬一有欄位缺失
    def apply_order(df, order):
        # 只選取 df 中實際存在的欄位，依照 order 排序
        existing_cols = [col for col in order if col in df.columns]
        return df[existing_cols]

    print("正在依指定順序重新排列欄位...")
    pivot_accuracy = apply_order(pivot_accuracy, final_column_order)
    pivot_macro_f1 = apply_order(pivot_macro_f1, final_column_order)
    pivot_off_by_1 = apply_order(pivot_off_by_1, final_column_order)
    overall_accuracy_df = apply_order(overall_accuracy_df, final_column_order)
    overall_off_by_1_df = apply_order(overall_off_by_1_df, final_column_order)


    # --- 8. 寫入 Excel 檔案 ---
    print(f"正在將所有結果寫入: {OUTPUT_EXCEL}")
    with pd.ExcelWriter(OUTPUT_EXCEL, engine='openpyxl') as writer:
        # completion_df 沒有維度欄位，不受影響
        completion_df.reset_index().to_excel(writer, sheet_name='Completion_Rate (完成率)', index=False)

        # 以下工作表將依照 final_column_order 順序寫入
        overall_accuracy_df.reset_index().to_excel(writer, sheet_name='Overall_Accuracy (總體準確率)', index=False)
        overall_off_by_1_df.reset_index().to_excel(writer, sheet_name='Overall_Off_by_1 (總體允許一級誤差)', index=False)
        pivot_accuracy.reset_index().to_excel(writer, sheet_name='Accuracy (完全一致率)', index=False)
        pivot_off_by_1.reset_index().to_excel(writer, sheet_name='Off_by_1_Accuracy (允許一級誤差)', index=False)
        pivot_macro_f1.reset_index().to_excel(writer, sheet_name='Macro_F1', index=False)

    print("\n---")
    print("分析成功完成！")
    print(f"所有結果已儲存至 Colab 暫存中的 '{OUTPUT_EXCEL}' 檔案。")
    print("請執行下一個 [儲存格 4] 以下載 Excel 結果檔案。")

except FileNotFoundError:
    print("\n---")
    print(f"錯誤：找不到檔案 '{INPUT_CSV}'。")
    print("請返回 [儲存格 2] 重新上傳檔案，然後再執行此儲存格。")
except Exception as e:
    print(f"\n分析過程中發生未預期的錯誤: {e}")

分析開始...
讀取資料: multidimensional_analysis_results_SLM+人工_不含理由_刪除無效_刪除評論_標準名稱_20251112.csv
基準 (人工標註) 評論總數: 400
正在計算完成率...
正在計算一致性指標 (Accuracy, F1, Off-by-1)...
正在計算總體 (懲罰性) 指標...
正在依指定順序重新排列欄位...
正在將所有結果寫入: model_consistency_analysis_results.xlsx

---
分析成功完成！
所有結果已儲存至 Colab 暫存中的 'model_consistency_analysis_results.xlsx' 檔案。
請執行下一個 [儲存格 4] 以下載 Excel 結果檔案。


In [ ]:
from google.colab import files
import os

OUTPUT_EXCEL = 'model_consistency_analysis_results.xlsx'

if os.path.exists(OUTPUT_EXCEL):
    print(f"正在準備下載 '{OUTPUT_EXCEL}'...")
    files.download(OUTPUT_EXCEL)
else:
    print(f"錯誤：找不到結果檔案 '{OUTPUT_EXCEL}'。")
    print("請先確認 [儲存格 3] 已成功執行完畢，並顯示「分析成功完成」。")

錯誤：找不到結果檔案 'model_consistency_analysis_results.xlsx'。
請先確認 [儲存格 3] 已成功執行完畢，並顯示「分析成功完成」。
